# Valentines Card Generator With Crew AI

This project is a simple demonstration of how to use the Crew AI API to generate a valentines card. The Crew AI API is a powerful tool that can be used to generate text, images, and more. In this project, we will use the Crew AI API to generate a valentines card with a custom message.

### Set up the environment 
Create a new environment and install the required packages.

```bash

conda create -n crew python=3.11

conda activate crew

```

In [2]:
# Install dependencies
! pip install crewai duckduckgo-search

  Using cached duckduckgo_search-4.4.3-py3-none-any.whl.metadata (19 kB)
  Using cached langchain_openai-0.0.2.post1-py3-none-any.whl.metadata (2.4 kB)
  Using cached openai-1.12.0-py3-none-any.whl.metadata (18 kB)
  Using cached opentelemetry_api-1.22.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.22.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pydantic-2.6.1-py3-none-any.whl.metadata (83 kB)
  Using cached regex-2023.12.25-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached curl_cffi-0.6.0b9-cp38-abi3-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached lxml-5.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached cffi-1.16.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.

In [29]:
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
from langchain_openai import ChatOpenAI


In [4]:
# Define the search tool 

search_tool = DuckDuckGoSearchRun()

In [17]:
# Define the topic 

topic = "Taylor Swift and long walks on the beach at sunset"

In [18]:
# Load a human 

human_tools = load_tools(['human'])

In [19]:
# Function that is run after the task is completed

def callback_function(output: TaskOutput):
    ## What to do after the task is completed (i.e. send an sms with a love poem)
    print(f"""
        Task completed with the following output:
        Task: {output.description}
        Output: {output.result}
    """)

In [20]:
# Custom tool 

class ContentTools:
    @tool("Read webpage content")
    def read_content(url: str) -> str:
        """Read the content of a webpage"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()
        return text_content[:5000]

### Create the Agents

In [21]:
# First agent: the love poem manager
manager = Agent(
    role = "Love Poem Manager",
    goal = "Find a love poem for my partner",
    verbose = True,
    backstory = "With a mind for romance, I want to deliver the best custom love poem to my client for Valentines Day",
    allow_delegation = True,
    max_iter = 10,
    max_rpm = 20,
)

In [22]:
# Second agent: the love poem searcher
researcher = Agent(
    role = "Love Poem Searcher",
    goal = f"Find a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a curious love poem searcher, and you want to 
    find the best and most custom love poem for your client to give to their partner""",
)

In [23]:
# Third agent: the love poem writer
writer = Agent(
    role = "Love Poem Writer",
    goal = f"Write a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a creative love poem writer, and you want to 
    write the best and most custom love poem for your client to give to their partner. 
    With a mind for romance and a deep love for poetry, you find the perfect poetry style 
    for your client and create a wonderful poem that conveys love, respect, kindness, and admiration""",
)

### Define the Tasks 

These are the research tasks that are needed to complete the project.

In [24]:
# Task 1: List ideas for a love poem

list_ideas = Task(
    description = "List of 5 cute ideas for a love poem, including possible poetry styles",
    expected_output = "Bullet point of 5 cute ideas for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

In [25]:
# Task 2: List important facts related to topic

list_facts = Task(
    description = f"List of 5 important facts specifically for someone that likes {topic}",
    expected_output = "Bullet point of 5 important facts related to the given topic for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

In [26]:
# Task 3: Write the love poem

write_poem = Task(
    description = f"Write a poem that would make Romeo and Juliet blush. The poem should be custom and tailored to the client's partner, who really likes {topic}",
    expected_output = "A lovely poem in a style that the client would like",
    tools=[search_tool],
    agent = writer,
    context=[list_ideas, list_facts],
    callback=callback_function,
)

In [27]:
# Task 4: Coordinate the love poem

manager_task = Task(
    description=f"""Oversee the creation of a great love poem for the client, 
    a lovebird who's looking for a custom poem for their partner, who really likes {topic}. 
    Ensure that the poem is custom and tailored to the client's partner and in a style that 
    is appropriate and conveys love, respect, kindness, and admiration""",
    expected_output=f"A lovely poem in a style that the client would like that incoporates the ideas and facts related to the {topic}",
    agent=manager,
)

### Form the Crew

In [30]:
crew = Crew(
    agents = [manager, researcher, writer],
    tasks = [list_ideas, list_facts, write_poem, manager_task],
    process = Process.hierarchical,
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),
)

Overriding of current TracerProvider is not allowed


### Run the Crew

In [31]:
result = crew.kickoff()

print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Delegate work to co-worker
Action Input: coworker: Love Poem Searcher, task: Find 5 important facts, context: We need to find 5 important facts specifically for someone that likes Taylor Swift and long walks on the beach at sunset. These facts will be used to create a love poem. The facts should be related to Taylor Swift's music, her personality, or her life that would resonate with someone who enjoys long walks on the beach at sunset. They should be romantic, inspiring, and should evoke a sense of love and affection. Thought: Do I need to use a tool? Yes
Action: Delegate work to co-worker
Action Input: coworker: Love Poem Writer, task: Generate 5 cute ideas for a love poem, context: We need to come up with 5 cute ideas for a love poem. This is a very important task and our job depends on it. The ideas should be unique and creative, and they should also

In [32]:
print(result)


Here is the poem titled "Swift Sunset Love":

In the golden twilight, where the sea meets the sky,
Hand in hand, we walk, as the sun begins to die.
Your laughter, a melody, echoing Taylor's sweet refrain,
Each sunset with you, a salve for life's mundane pain.

Through the rhythmic waves, our love story unfolds,
As enchanting and timeless, as the classics of old.
Under the canvas of the dusk, painted in hues so bright,
Our shadows dance, in the soft and fading light.

In the silence, your gaze speaks volumes to me,
Like a love-struck Romeo, beneath Juliet's balcony.
In your eyes, I see a love, strong and unwavering,
Like a lighthouse, guiding me, forever anchoring.

Our footprints in the sand, a testament to our journey,
A tale of enduring love, of you and me.
As the sun dips low, surrendering to the night,
Our hearts echo Taylor's words, "you are in love, right?"

With each wave that kisses the shore, with the moon up above,
We are reminded of Swift's tales, of longing and love.
In the